Load in the new hourly counts dataset:

In [4]:
import numpy as np
import pandas as pd

np.random.seed(42)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# I had to run this in Google Colab for the neural net.

# Here is the correct path if you are running it locally:
#df = pd.read_csv('../data/hourlycalls_weather_traffic.csv')
df = pd.read_csv('/content/drive/MyDrive/DSI/hourlycalls_weather_traffic.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25465 entries, 0 to 25464
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                25465 non-null  int64  
 1   Unnamed: 0.1              25465 non-null  int64  
 2   year                      25465 non-null  int64  
 3   month                     25465 non-null  int64  
 4   day                       25465 non-null  int64  
 5   hour                      25465 non-null  int64  
 6   num_calls                 25465 non-null  int64  
 7   BRONX                     25465 non-null  int64  
 8   BROOKLYN                  25465 non-null  int64  
 9   MANHATTAN                 25465 non-null  int64  
 10  QUEENS                    25465 non-null  int64  
 11  RICHMOND / STATEN ISLAND  25465 non-null  int64  
 12  UNKNOWN                   25465 non-null  int64  
 13  STATION                   25465 non-null  object 
 14  NAME  

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn tools:
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
# sklearn models:
from sklearn.linear_model import LinearRegression
# tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input


Set up X and y, do train test split, and scale:

In [8]:
X = df[['year', 'month', 'day', 'hour', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TAVG_CALC']]
y = df['num_calls']
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)
sc = StandardScaler()
Z_train = sc.fit_transform(X_train)
Z_test = sc.transform(X_test)

Baseline:

In [9]:
m = y_train.mean()
baseline_train = [m for y in y_train]
baseline_test  = [m for y in y_test]
R20 = r2_score(y_train,baseline_train)
R21 = r2_score(y_test,baseline_test)
RMS0= mean_squared_error(y_train,baseline_train,squared=False)
RMS1= mean_squared_error(y_test, baseline_test, squared=False)
print(f'BASELINE, call volume = {m}')
print(f'R2:  Train: {R20}, Test: {R21}')
print(f'RMS: Train: {RMS0}, Test: {RMS1}')

BASELINE, call volume = 136.38548539114043
R2:  Train: 0.0, Test: -1.7549351112977618e-05
RMS: Train: 54.20031558657461, Test: 54.340338222301554


Linear Modeling:

In [10]:
lr1 = LinearRegression()
lr1.fit(X_train,y_train)
R20 = lr1.score(X_train,y_train)
R21 = lr1.score(X_test,y_test)
RMS0= mean_squared_error(y_train,lr1.predict(X_train),squared=False)
RMS1= mean_squared_error(y_test, lr1.predict(X_test), squared=False)
print(f'R2:  Train: {R20}, Test: {R21}')
print(f'RMS: Train: {RMS0}, Test: {RMS1}')

R2:  Train: 0.47955354948268436, Test: 0.478650922673906
RMS: Train: 39.10117792607335, Test: 39.23582881769342


We know several features have non-linear interactions with num_calls. Let's try Polynomial Features:

In [11]:
for n in range(2,5):
    print()
    print(f'Polynomial Features of Degree {n}:')
    pf = PolynomialFeatures(degree=n)
    PF_train = pf.fit_transform(Z_train)
    PF_test  = pf.transform(Z_test)
    lr2 = LinearRegression()
    lr2.fit(PF_train,y_train)
    R20 = lr2.score(PF_train,y_train)
    R21 = lr2.score(PF_test,y_test)
    RMS0= mean_squared_error(y_train,lr2.predict(PF_train),squared=False)
    RMS1= mean_squared_error(y_test, lr2.predict(PF_test), squared=False)
    print(f'R2:  Train: {R20}, Test: {R21}')
    print(f'RMS: Train: {RMS0}, Test: {RMS1}')


Polynomial Features of Degree 2:
R2:  Train: 0.5323707756578696, Test: 0.5294029037291601
RMS: Train: 37.064029894825815, Test: 37.27718834630224

Polynomial Features of Degree 3:
R2:  Train: 0.7224120216530114, Test: 0.7193914502514452
RMS: Train: 28.556313573505026, Test: 28.78518175805514

Polynomial Features of Degree 4:
R2:  Train: 0.777497979167833, Test: 0.7328983505400335
RMS: Train: 25.566353830792764, Test: 28.08386077922705


Polynomial Features definitely helped!

Neural Net, first attempt:

In [12]:
#set random state for reproducability:

model = Sequential()
model.add(Dense(32, input_dim=Z_train.shape[1], 
                    activation='relu'))
model.add(Dense(16,  
                    activation='relu'))
model.add(Dense(8,  
                    activation='relu',))
model.add(Dense(1, activation=None))

model.compile(loss='mse', optimizer='adam', metrics=['mae'])


results = model.fit(Z_train, y_train, epochs=100, batch_size=256, \
                            validation_data=(Z_test, y_test))



Epoch 1/100
75/75 [==============================] - 1s 8ms/step - loss: 21350.8640 - mae: 135.6819 - val_loss: 20292.1094 - val_mae: 131.8097
Epoch 2/100
75/75 [==============================] - 0s 2ms/step - loss: 19251.3517 - mae: 127.8479 - val_loss: 13134.4785 - val_mae: 102.4658
Epoch 3/100
75/75 [==============================] - 0s 3ms/step - loss: 10092.0454 - mae: 86.2307 - val_loss: 3388.9895 - val_mae: 46.0008
Epoch 4/100
75/75 [==============================] - 0s 2ms/step - loss: 3036.0381 - mae: 43.8200 - val_loss: 2589.6782 - val_mae: 40.6767
Epoch 5/100
75/75 [==============================] - 0s 3ms/step - loss: 2450.2496 - mae: 39.4252 - val_loss: 2309.7766 - val_mae: 38.5717
Epoch 6/100
75/75 [==============================] - 0s 3ms/step - loss: 2259.4903 - mae: 38.1977 - val_loss: 2150.3691 - val_mae: 37.3348
Epoch 7/100
75/75 [==============================] - 0s 2ms/step - loss: 2067.2870 - mae: 36.5209 - val_loss: 2039.1487 - val_mae: 36.4663
Epoch 8/100
75/75 

In [13]:
preds_nn0 = model.predict(Z_train)
preds_nn1 = model.predict(Z_test)

In [14]:
R20 = lr2.score(PF_train,y_train)
R21 = lr2.score(PF_test,y_test)
RMS0= mean_squared_error(y_train,lr2.predict(PF_train),squared=False)
RMS1= mean_squared_error(y_test, lr2.predict(PF_test), squared=False)
print('Best Linear Model:')
print(f'R2:  Train: {R20}, Test: {R21}')
print(f'RMS: Train: {RMS0}, Test: {RMS1}')

Best Linear Model:
R2:  Train: 0.777497979167833, Test: 0.7328983505400335
RMS: Train: 25.566353830792764, Test: 28.08386077922705


In [15]:
R20 = r2_score(y_train,preds_nn0)
R21 = r2_score(y_test, preds_nn1)
RMS0= mean_squared_error(y_train,preds_nn0,squared=False)
RMS1= mean_squared_error(y_test, preds_nn1, squared=False)
print('Neural Net Model:')
print(f'R2:  Train: {R20}, Test: {R21}')
print(f'RMS: Train: {RMS0}, Test: {RMS1}')

Neural Net Model:
R2:  Train: 0.7878203250078152, Test: 0.7927309775100686
RMS: Train: 24.966272429894985, Test: 24.739204241559914


So there is definitely some room for improvement, and we should be able to get a good model from this data.